In [6]:
#import statements, import data
import pandas as pd
import numpy as np
import funcs
%load_ext autoreload
%autoreload 2

castile_raw = pd.read_excel('../data/climate data.xlsx', sheet_name = 'castile')
castile_area = 118104 #ft^2
castile_raw = castile_raw.replace(to_replace='T', value = 0) #get rid of trace values from NOAA data
castile_raw= funcs.convert_units(castile_raw) #change units

dftest = pd.read_excel('../data/climate data.xlsx', sheet_name = 'castile')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
castile_mb = funcs.mass_bal(castile_raw, castile_area)

In [19]:
print(castile_mb['Pin[mm]'].sum())

246.88


In [3]:
print(castile_raw.iloc[0:7, 1:].sum()/7)
print(castile_raw.iloc[0:7, 1:].mean())
print(castile_raw.iloc[0:7, 1:])

test = castile_raw.iloc[0:7, 1:].mean()
test2 = pd.concat([pd.Series(data =[5], index =['date']),test])
dftest.loc[len(dftest)] = test2


days = dftest.shape[0]
wks = int((days - days % 7) / 7) #only looking at full weeks
dfwk = pd.DataFrame(columns=dftest.columns)
dfwk = dfwk.rename(columns={'date': 'week'})
print(test2[0])


P[mm]       4.644286
Tavg[C]     6.112857
Rhavg      77.157143
dtype: float64
P[mm]       4.644286
Tavg[C]     6.112857
Rhavg      77.157143
dtype: float64
   P[mm]  Tavg[C]  Rhavg
0   8.13     7.78   78.9
1   7.11     3.89   82.3
2   3.05     2.78   86.3
3   5.33     3.89   86.1
4   8.89     5.28   73.3
5   0.00     8.06   61.7
6   0.00    11.11   71.5
5.0


In [12]:
Pin = castile_raw.iloc[0:7, 1].sum() #cumulative precipitation for wk [mm/wk]
avgs = castile_raw.iloc[0:7, 2:].mean() #weekly averages for equation
Eout = 100 * (0.135 * avgs[0] - 0.024 * Pin - 0.099 * avgs[1] + 7.14) #equation from paper for raw manure evaporation, converted to [mm/wk]
net = (Pin - Eout) * 0.00328 * castile_area #net water volume into system [mm^3]
pcost = net * 7.481 * 0.01

row = pd.Series([1, Pin, Eout, net, pcost], index = ['week', 'Pin[mm]', 'Eout[mm]', 'Net Change[mm^3]', 'Pumping Cost [$]']) #row to add to dfmb